<a href="https://colab.research.google.com/github/codeworks-data/mvp-dpe/blob/main/notebooks/temperatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#For data manipulation
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import groupby


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
data = pd.read_csv('/content/drive/My Drive/datasets/temperature-quotidienne-departementale.csv', sep=';')

In [7]:
data.head(5)

,date_obs,code_insee_departement,departement,tmin,tmax,tmoy
0,2020-01-01,43,Haute-Loire,-5.00,9.20,2.10
1,2020-01-01,51,Marne,-2.75,2.60,-0.08
2,2020-01-01,58,Nièvre,0.40,5.80,3.10
3,2020-01-01,63,Puy-de-Dôme,-4.30,9.80,2.75
4,2020-01-01,50,Manche,5.51,8.59,7.05


In [8]:
data.dtypes

date_obs                   object
code_insee_departement     object
departement                object
tmin                      float64
tmax                      float64
tmoy                      float64
dtype: object

In [9]:
data['date_obs'] = pd.to_datetime(data['date_obs'])

In [15]:
LOOKUP_SEASON = {
    12: 'Winter',
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Autumn',
    10: 'Autumn',
    11: 'Autumn'
}

In [16]:
def get_season(date):
    season = LOOKUP_SEASON[date.month]
    return season

In [17]:
data['season'] = data['date_obs'].apply(lambda x: get_season(x))

In [18]:
data.head(3)

,date_obs,code_insee_departement,departement,tmin,tmax,tmoy,season
0,2020-01-01,43,Haute-Loire,-5.00,9.2,2.10,Winter
1,2020-01-01,51,Marne,-2.75,2.6,-0.08,Winter
2,2020-01-01,58,Nièvre,0.40,5.8,3.10,Winter


In [19]:
data.groupby(['code_insee_departement', 'season'])['tmin','tmax','tmoy'].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tmin       tmax       tmoy
code_insee_departement season                                 
01                     Autumn   7.685348  17.923700  12.804505
                       Spring   5.912681  17.972464  11.942572
                       Summer  14.221377  28.169964  21.195688
                       Winter   1.164238   8.421623   4.792914
02                     Autumn   7.796337  16.000623  11.898498
...                                  ...        ...        ...
94                     Winter   3.387748   8.884238   6.135993
95                     Autumn   8.834689  17.052418  12.943810
                       Spring   6.829022  16.909275  11.868913
                       Summer  14.542717  26.128623  20.335362
                       Winter   3.159272   8.789272   5.974503

[384 rows x 3 columns]

In [20]:
seasons = data.groupby(['code_insee_departement', 'season'])['tmin','tmax','tmoy'].mean().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [23]:
seasons

,code_insee_departement,season,tmin,tmax,tmoy
0,01,Autumn,7.685348,17.923700,12.804505
1,01,Spring,5.912681,17.972464,11.942572
2,01,Summer,14.221377,28.169964,21.195688
3,01,Winter,1.164238,8.421623,4.792914
4,02,Autumn,7.796337,16.000623,11.898498
...,...,...,...,...,...
379,94,Winter,3.387748,8.884238,6.135993
380,95,Autumn,8.834689,17.052418,12.943810
381,95,Spring,6.829022,16.909275,11.868913
382,95,Summer,14.542717,26.128623,20.335362


In [33]:
df = pd.pivot_table(seasons,index=['code_insee_departement'], columns=seasons.groupby(['code_insee_departement']).cumcount().add(1), values=['tmin','tmax', 'tmoy']).reset_index()

In [34]:
df

code_insee_departement       tmax             ...       tmoy                     
                                   1          2  ...          2          3         4
0                      01  17.923700  17.972464  ...  11.942572  21.195688  4.792914
1                      02  16.000623  16.041304  ...  10.886413  19.016304  4.988046
2                      03  19.000733  18.093841  ...  11.624094  20.711739  5.608808
3                      04  20.043993  18.816667  ...  12.680616  22.871051  6.196060
4                      05  18.373150  17.308696  ...  11.130435  20.835362  3.465894
..                    ...        ...        ...  ...        ...        ...       ...
91                     91  17.361538  17.239130  ...  12.338043  21.097500  6.135993
92                     92  16.374212  16.278696  ...  11.818623  20.357210  5.644040
93                     93  17.312161  17.176087  ...  12.257464  20.842246  6.265596
94                     94  17.361538  17.239130  ...  12.338043  21.097500  6.135993
95                     95  17.052418  16.909275  ...  11.868913  20.335362  5.974503

[96 rows x 13 columns]

In [35]:
df.columns = df.columns.map('{0[0]}_{0[1]}'.format) 

In [36]:
df

,code_insee_departement_,tmax_1,tmax_2,tmax_3,tmax_4,tmin_1,tmin_2,tmin_3,tmin_4,tmoy_1,tmoy_2,tmoy_3,tmoy_4
0,01,17.923700,17.972464,28.169964,8.421623,7.685348,5.912681,14.221377,1.164238,12.804505,11.942572,21.195688,4.792914
1,02,16.000623,16.041304,25.057283,7.740000,7.796337,5.731522,12.975362,2.236093,11.898498,10.886413,19.016304,4.988046
2,03,19.000733,18.093841,28.138659,9.740430,7.344322,5.154348,13.284783,1.477152,13.172491,11.624094,20.711739,5.608808
3,04,20.043993,18.816667,30.026594,10.942748,9.059341,6.544565,15.715580,1.449338,14.551648,12.680616,22.871051,6.196060
4,05,18.373150,17.308696,28.080471,8.483808,7.076557,4.952174,13.590217,-1.551987,12.724835,11.130435,20.835362,3.465894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,17.361538,17.239130,26.758804,8.884238,9.312088,7.436957,15.436232,3.387748,13.336813,12.338043,21.097500,6.135993
92,92,16.374212,16.278696,25.517355,8.199437,9.204872,7.358659,15.197138,3.088675,12.789560,11.818623,20.357210,5.644040
93,93,17.312161,17.176087,26.291957,9.081291,9.277289,7.338732,15.392609,3.449834,13.294725,12.257464,20.842246,6.265596
94,94,17.361538,17.239130,26.758804,8.884238,9.312088,7.436957,15.436232,3.387748,13.336813,12.338043,21.097500,6.135993


In [37]:
NAMES_SEASONS = {
'1': 'Autumn',
'2': 'Spring',
'3':'Summer',
'4':'Winter'
}

In [38]:
for key in NAMES_SEASONS.keys():
    df.columns = df.columns.str.replace(key, NAMES_SEASONS[key])

In [39]:
df

,code_insee_departement_,tmax_Autumn,tmax_Spring,tmax_Summer,tmax_Winter,tmin_Autumn,tmin_Spring,tmin_Summer,tmin_Winter,tmoy_Autumn,tmoy_Spring,tmoy_Summer,tmoy_Winter
0,01,17.923700,17.972464,28.169964,8.421623,7.685348,5.912681,14.221377,1.164238,12.804505,11.942572,21.195688,4.792914
1,02,16.000623,16.041304,25.057283,7.740000,7.796337,5.731522,12.975362,2.236093,11.898498,10.886413,19.016304,4.988046
2,03,19.000733,18.093841,28.138659,9.740430,7.344322,5.154348,13.284783,1.477152,13.172491,11.624094,20.711739,5.608808
3,04,20.043993,18.816667,30.026594,10.942748,9.059341,6.544565,15.715580,1.449338,14.551648,12.680616,22.871051,6.196060
4,05,18.373150,17.308696,28.080471,8.483808,7.076557,4.952174,13.590217,-1.551987,12.724835,11.130435,20.835362,3.465894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,17.361538,17.239130,26.758804,8.884238,9.312088,7.436957,15.436232,3.387748,13.336813,12.338043,21.097500,6.135993
92,92,16.374212,16.278696,25.517355,8.199437,9.204872,7.358659,15.197138,3.088675,12.789560,11.818623,20.357210,5.644040
93,93,17.312161,17.176087,26.291957,9.081291,9.277289,7.338732,15.392609,3.449834,13.294725,12.257464,20.842246,6.265596
94,94,17.361538,17.239130,26.758804,8.884238,9.312088,7.436957,15.436232,3.387748,13.336813,12.338043,21.097500,6.135993


In [40]:
df.rename(columns={"code_insee_departement_":'code_departement'}, inplace=True) 

In [41]:
df.to_csv('/content/drive/My Drive/datasets/temperatures.csv', index = 0)